Using https://www.kaggle.com/cyannani123/keras-cellular-image-classification as test example

In [1]:
import os
import sys
import numpy as np
import pandas as pd
import tensorflow as tf
import joblib

In [2]:
from tqdm import tqdm

In [3]:
sys.path.append('../src')

In [4]:
from data_v2 import augment
from data_v2 import get_center_box
from data_v2 import get_random_subbox
from data_v2 import get_image, get_channels
from data_v2 import ImgGen

In [5]:
img=get_channels('RPE-01','1','B02','1',"../data/train/", lambda x:x)
img.shape


(6, 512, 512)

Loading test and train data.

Will load directly from zip for now.

In [5]:
train_data_all = pd.read_csv("../data/train.csv")
print("Shape of train_data_all:", train_data_all.shape)


Shape of train_data_all: (36517, 5)


In [6]:
train_data_all[['cell_line','exp']]=list(train_data_all.experiment.str.split('-'))

Võtame esialgu ainul HUVEC liini

In [7]:
train_data_all=train_data_all.loc[train_data_all.cell_line=='HUVEC',:]

In [8]:
train_data_all.head()

,id_code,experiment,plate,well,sirna,cell_line,exp
7750,HUVEC-01_1_B03,HUVEC-01,1,B03,sirna_403,HUVEC,01
7751,HUVEC-01_1_B04,HUVEC-01,1,B04,sirna_348,HUVEC,01
7752,HUVEC-01_1_B05,HUVEC-01,1,B05,sirna_678,HUVEC,01
7753,HUVEC-01_1_B06,HUVEC-01,1,B06,sirna_39,HUVEC,01
7754,HUVEC-01_1_B07,HUVEC-01,1,B07,sirna_355,HUVEC,01


In [17]:
train_data_strat=train_data_all.groupby('sirna',group_keys=False).apply(lambda x: x.sample(frac=0.1).reset_index(drop=True))

In [18]:
len(train_data_strat)

2214

In [19]:
train_data_strat.reset_index(drop=True,inplace=True)

In [21]:
with tf.io.TFRecordWriter("../data/processed/train_data_strat.tfrecord") as writer:

    for exp, plate, well, sirna in tqdm(train_data_strat.loc[:,("experiment","plate","well","sirna")].values.tolist()):
        #print(exp, plate, well, sirna)
        img = get_image(exp, plate, well,1,"../data/train.zip", lambda x: x)
        #print(img.shape)

        label = int(sirna.split('_')[1])

        # write image and sirna label string to tfrecord
        img_raw = img.tobytes()
        example = tf.train.Example(features=tf.train.Features(feature={
            'image': tf.train.Feature(bytes_list=tf.train.BytesList(value=[img_raw])),
            'label': tf.train.Feature(int64_list=tf.train.Int64List(value=[label]))
        }))
        writer.write(example.SerializeToString())


  4%|▍         | 89/2214 [03:32<1:24:31,  2.39s/it]


KeyboardInterrupt: decoding with 'cp437' codec failed (KeyboardInterrupt: )